We will use the following context free grammar structure:
```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token: (class, value)
class: int
value: str
var: str                 # name of the non-terminal symbol
definition: list(rule)
rule: list(var | token)  # right part of the rule
```
Note: the our grammar has only productive and reacheable symbols. Also the grammar has not left recursion.

In [1]:
def vanishing_symbols(grammar) -> set:
  """Find all vanishing symbols of the given grammar."""
  
  vanishing_symbols = set()

  prev_number = None
  while prev_number != len(vanishing_symbols):
    prev_number = len(vanishing_symbols)
    for nonterminal, definition in grammar['vars'].items():
      for rule in definition:
        if all([s in vanishing_symbols or s == (0, '') for s in rule]):
          vanishing_symbols.add(nonterminal)

  return vanishing_symbols  

FIRST function returns set of terminals symbols that strings derived from A (that we passed to it) begin with.

First(A) for each rule from non-terminal A: 
 - for first vanishing symbols E => add to set First(E) to the set
 - if the first symbol after vanishing symbols is terminal => add this token to the set
 - if the first symbol after vanishing symbols is non-terminal B => add First(B) to the set

If the A is non-terminal - we add empty token to the set

In [10]:
def first(grammar, nonterminal) -> set:
  nonterminal_first = set()
  for rule in grammar['vars'][nonterminal]:
    # add FIRST for first vanishing symbols
    index = 0
    while index < len(rule) and rule[index] in vanishing_symbols(grammar):
      nonterminal_first = nonterminal_first.union(first(grammar, rule[index]))
      index += 1

    # add the first terminal symbol after all vanishing symbols
    if index < len(rule) and rule[index] in grammar['toks']:
      nonterminal_first.add(rule[index])

    # add the FIRST of the nonterminal symbol after all vanishing symbols
    if index < len(rule) and rule[index] not in grammar['toks']:
      nonterminal_first = nonterminal_first.union(first(grammar, rule[index]))

  # add empty token if the non-terminal is vanishing symbol
  if nonterminal in vanishing_symbols(grammar):
    nonterminal_first.add((0, ''))
  return nonterminal_first

FOLLOW function returns set of terminals that can appear after non-terminal A during deduction.

FOLLOW(A) for each rule:
 1. find all occurrences of a given non-terminal A
 2. for each occurrences: 
  - for first vanishing symbols E after the given occurrence of A => add to set First(E) to the set
  - if the first symbol after vanishing symbols is terminal => add this token to the set
  - if the first symbol after vanishing symbols is non-terminal B => add First(B) to the set
  - if all symbols after the given occurrence of A are vanishing symbols => 
      - add FOLLOW(B) to the set, where B is the symbol of the left part of the rule
      - we add empty token to the set.

In [18]:
def follow(grammar, nonterminal) -> set:
  """Find set of terminals that can appear after the giben non-terminal A during deduction"""
  nonterminal_follow = set()
  for var, definitions in grammar['vars'].items():
    for rule in definitions:
      for i in [index for index, symbol in enumerate(rule) if symbol == nonterminal]:
        # add FIRST of each vanish symbol that follows after the given nonterminal symbol
        index = i + 1
        while index < len(rule) and rule[index] in vanishing_symbols(grammar):
          nonterminal_follow = nonterminal_follow.union(first(grammar, rule[index]))
          index += 1

        # add the first terminal symbol after all vanishing symbols
        if index < len(rule) and rule[index] in grammar['toks']:
          nonterminal_follow.add(rule[index])

        # add the FIRST of the nonterminal symbol after all vanishing symbols
        if index < len(rule) and rule[index] not in grammar['toks']:
          nonterminal_follow = nonterminal_follow.union(first(grammar, rule[index]))

        # add FOLLOW of var and empty symbol
        if index >= len(rule):
          nonterminal_follow = nonterminal_follow.union(follow(grammar, var)) if var != nonterminal else nonterminal_follow
          nonterminal_follow.add((0, ''))
  return nonterminal_follow

In [26]:
grammar = {
    'toks': {(0, ''), (1, '+'), (2, '*'), (3, '('), (4, ')'), (5, 'v'), (6, 'n')},
    'vars': {
        'S': [['T', "S'"]], 
        "S'": [[(1, '+'), 'T', "S'"], [(0, '')]], 
        'T': [['F', "T'"]],
        "T'": [[(2, '*'), 'F', "T'"], [(0, '')]],
        'F': [[(5, 'v')], [(6, 'n')], [(3, '('), 'S', (4, ')')]]
        },
    'hvar': 'S'
    }

print(first(grammar, 'S') == {(5, 'v'), (6, 'n'), (3, '(')})
print(first(grammar, "S'") == {(1, '+'), (0, '')})
print(first(grammar, 'T') == {(5, 'v'), (6, 'n'), (3, '(')})
print(first(grammar, "T'") == {(2, '*'), (0, '')})
print(first(grammar, 'F') == {(5, 'v'), (6, 'n'), (3, '(')})

print(follow(grammar, 'S') == {(4, ')')})
print(follow(grammar, "S'") == {(4, ')'), (0, '')})
print(follow(grammar, 'T') ==  {(1, '+'), (4, ')'), (0, '')})
print(follow(grammar, "T'") == {(1, '+'), (4, ')'), (0, '')})
print(follow(grammar, "F") == {(1, '+'), (2, '*'), (4, ')'), (0, '')})

True
True
True
True
True
True
True
True
True
True
